In [ ]:
# IMPORTS

# Librerías principales, para tratar imágenes, desarrollar la red neuronal, tratar datos, etcétera
import os
import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import keras

# Librería para pintar histogramas
import plotly.express as px

# FROMS

# Hace algo importante
from glob import glob

# Para hacer sortcuts a los xrange() de los bucles
from tqdm import tqdm

# Todo lo que necesitamos de la librería sklearn
from sklearn.model_selection import train_test_split      
from keras.utils import np_utils

# Todo lo que necesitamos de la librería keras-tensorflow
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Declaración de funciones

### Sacamos la imagen

In [ ]:
def sacar_imagen(path, img_filas, img_columnas):
    # Cargamos la imagen que se encuentra en el path con los colores RGB
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    # Redimensionamos la imagen al tamaño que queremos
    img = cv2.resize(img, (img_filas, img_columnas)) 
    return img

### Cargamos el entrenamiento

In [ ]:
def cargar_entrenamiento(img_filas, img_columnas):
    # Inicializamos las variables
    imagenes_entrenamiento, id_entrenamiento = [], []
    # Hacemos un bucle que se va ha ejecutar num_clases veces
    for clases in tqdm(range(num_clases)):
        # Leemos la carpeta de donde sacaremos las imagenes que vamos a utilizar en el entrenamiento
        fichero = glob(os.path.join('../input/state-farm-distracted-driver-detection/imgs/train/c' + str(clases), '*.jpg'))
        # Hacemos un bucle que recorre toda la carpeta de imágenes
        for img in fichero:
            # Sacamos una imagen, utilizando la función que hemos creado
            ig = sacar_imagen(img, img_filas, img_columnas)
            # Almacenamos la imagen
            imagenes_entrenamiento.append(ig)
            # Almacenamos la clase
            id_entrenamiento.append(clases)
    return imagenes_entrenamiento, id_entrenamiento

### Leer y normalizar los datos de entrenamiento

In [ ]:
def LyN_datos_entrenamiento(img_filas, img_columnas):
    # Cargamos el entrenamiento (entrada y salida)
    X, etiquetas = cargar_entrenamiento(img_filas, img_columnas)
    # Convertimos nuestra salida de un vector de número enteros a una matriz de clase binaria
    y = np_utils.to_categorical(etiquetas, num_clases)
    # Dividimos los datos de E/S en: set de entrenamiento y set de test
    entrada_entrenamiento, entrada_validacion, salida_entrenamiento, salida_validacion = train_test_split(X, y, test_size=0.2, random_state=42)
    # Hacemos que la entrada del entrenamiento y del test sean una numpy array y las reescalamos
    entrada_entrenamiento = np.array(entrada_entrenamiento, dtype=np.uint8).reshape(-1, img_filas, img_columnas, 3)
    entrada_validacion = np.array(entrada_validacion, dtype=np.uint8).reshape(-1, img_filas, img_columnas, 3)
    return entrada_entrenamiento, entrada_validacion, salida_entrenamiento, salida_validacion

### Cargamos el test

In [ ]:
def cargar_test(size, img_filas, img_columnas):
    # Cargamos el path del test
    path = os.path.join('../input/state-farm-distracted-driver-detection/imgs/test', '*.jpg')
    ficheros = sorted(glob(path))
    # Declaramos las variables
    entrada_evaluacion, id_entrada_evaluacion = [], []
    total = 0
    # Vemos cuantas imágenes hay en la dirección del test
    size_fichero = len(ficheros)
    # Hacemos un bucle, que recorre todas las imágenes una a una
    for fichero in tqdm(ficheros):
        # Si total es mayor o igual a size o a la cantidad de imágenes que hay
        if total >= size or total >= size_fichero:
            # En caso afirmativo: 
            break
        base_fichero = os.path.basename(fichero)
        # Sacamos la imagen
        img = sacar_imagen(fichero, img_filas, img_columnas)
        # Almacenamos la imagen
        entrada_evaluacion.append(img)
        id_entrada_evaluacion.append(base_fichero)
        # Incrementamos el total
        total+=1
    return entrada_evaluacion, id_entrada_evaluacion

### Leer y normalizar los datos de test

In [ ]:
def LyN_datos_test(size, img_filas, img_columnas):
    # Cargamos las imagenes del test y sus id's
    datos_test, ids_test = cargar_test(size, img_filas, img_columnas)
    # Hacemos que los datos para el test, sean una numpy array y la reescalamos
    datos_test = np.array(datos_test, dtype=np.uint8).reshape(-1, img_filas, img_columnas, 3)
    return datos_test, ids_test

## Declaración de variables globales

In [ ]:
# 0: practica_1 | 1: practica_2
dato = 1
img_filas = 128
img_columnas = 98
num_test_examples = 200

## Declaración del dataset

In [ ]:
dataset = pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')

if dato == 0:
    num_clases = len(dataset['classname'].unique())
else:
    num_clases = len(dataset['subject'].unique())

## Cargamos las imágenes de entrenamiento

In [ ]:
# Sacamos la E/S del entrenamiento y el test, ya normalizados
entrada_entrenamiento, entrada_validacion, salida_entrenamiento, salida_validacion = LyN_datos_entrenamiento(img_filas, img_columnas)

## Cargamos las imágenes de validación

In [ ]:
# Sacamos la E/S de la validación, ya normalizado
entrada_evaluacion, salida_evaluacion = LyN_datos_test(num_test_examples, img_filas, img_columnas)

## Datos (Visualización)

### Histograma de Categorias 

In [ ]:
# Pintamos el histograma
px.histogram(dataset, x='classname', color='classname', title='Categorias')

### Histograma de Conductores

In [ ]:
# Generamos un dataframe propio para los conductores 
conductores_id = pd.DataFrame((dataset['subject'].value_counts()).reset_index())
conductores_id.columns = ['id_conductor', 'cantidad']
# Pintamos el histograma
px.histogram(conductores_id, x='id_conductor', y='cantidad', color='id_conductor', title='Conductores')

## CNN (Convolution Neural Network)

In [ ]:
# Inicializamos el modelo
modelo = Sequential()

# CNN 1 - Input
modelo.add(Conv2D(64,(3,3),activation='relu',input_shape=(img_filas, img_columnas, 3)))
modelo.add(BatchNormalization())
modelo.add(Conv2D(64,(3,3),activation='relu'))
modelo.add(BatchNormalization(axis = 3))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Dropout(0.5))

# CNN 2
modelo.add(Conv2D(64,(3,3),activation='relu'))
modelo.add(BatchNormalization())
modelo.add(Conv2D(64,(3,3),activation='relu'))
modelo.add(BatchNormalization(axis = 3))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Dropout(0.5))

# CNN 3
modelo.add(Conv2D(128,(3,3),activation='relu'))
modelo.add(BatchNormalization())
modelo.add(Conv2D(128,(3,3),activation='relu'))
modelo.add(BatchNormalization(axis = 3))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Dropout(0.5))

# Output
modelo.add(Flatten())
modelo.add(Dense(512,activation='relu'))
modelo.add(BatchNormalization())
modelo.add(Dropout(0.5))
modelo.add(Dense(512,activation='relu'))
modelo.add(Dropout(0.25))
modelo.add(Dense(num_clases,activation='softmax'))

In [ ]:
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = modelo.fit(entrada_entrenamiento, salida_entrenamiento, validation_data=(entrada_validacion, salida_validacion), epochs=10, batch_size=40, verbose=1)

In [ ]:
evaluacion = modelo.evaluate(entrada_validacion, salida_validacion, verbose=1)
print('Error: {:.2f}'.format(evaluacion[0]))
print('Precisión: {:.2f}%'.format(evaluacion[1]*100))

In [ ]:
result = modelo.predict(entrada_evaluacion)
for i in result:
    print('----------------------------------')
    if dato == 0:
        accion = dataset['classname'].unique()
        porcentajes = [i[k]*100 for k in range(len(i))]
        plt.pie(porcentajes, labels=accion, autopct="%0.2f %%")
        plt.axis("equal")
        plt.show()
    else:
        conductores = dataset['subject'].unique()
        porcentajes = [i[k]*100 for k in range(len(i))]
        plt.pie(porcentajes, labels=conductores, autopct="%0.2f %%")
        plt.axis("equal")
        plt.show()